# 1. 이진분류_콘크리트크랙

## (1) import

In [ ]:
from glob import glob

import zipfile
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 전처리, 학습
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## (2) 데이터 확인

In [ ]:
forlder_path = ''

##### 필요할 진 모르겠는 코드..

In [ ]:
with zipfile.ZipFile(folder_path + '...images.zip', 'r') as zip_ref:
    zip_ref.extractall(folder_path)

In [ ]:
# train/test 이미지가 들어있는 폴더의 경로
train = '...images/train'
test = '...images/test'

In [ ]:
# 이미지가 분류되어 있지 않고, 파일명에 기입되어 있는 경우 사용하는 코드
if not os.path.exists('IMAGE'):
    os.mkdir('IMAGE')
if not os.path.exists('IMAGE/Negative'):
    os.mkdir('IMAGE/Negative')
if not os.path.exists('IMAGE/Positive'):
    os.mkdir('IMAGE/Positive')

count = 0
# 파일명 예시: 0_0_0_20161219140623097_Negative.jpg or 0_0_0_20161219140523931_Positive.jpg
for filename in os.listdir(train):
    if filename.endswith('.jpg'):
        label = filename.split("_")[-1].split(".")[0]
        
        if label == "Negative":
            # 현재 폴더에서 Negative 폴더로 파일 복사
            shutil.copy(os.path.join(train, filename), os.path.join('IMAGE/Negative', filename))
        elif label == "Positive":
            # 현재 폴더에서 Positive 폴더로 파일 복사
            shutil.copy(os.path.join(train, filename), os.path.join('IMAGE/Positive', filename))

In [ ]:
# 이미지 개수 확인 !! 폴더 경로 수정 필요
!ls -l ./IMAGE/Negative/ | grep jpg | wc -l

In [ ]:
# 이미지 개수 확인 !! 폴더 경로 수정 필요
!ls -l ./IMAGE/Positive/ | grep jpg | wc -l

In [ ]:
# 이미지 파일 하나 확인
path = 'IMAGE/Positive/' + os.listdir('IMAGE/Positive')[1]
print('path:', path)

In [ ]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype=tf.float32)
print('image.shape:', image.shape)

In [ ]:
plt.imshow(image)
plt.show()

## (3) 데이터 전처리

In [ ]:
# ImageDataGenerator: 이미지 데이터에 대해 scaling, augmentation, validattion_split 전처리 기능 수행
# flow_from_directory: 실제 이미지 데이터를 읽고 배치, 셔플하고 labeling 수행

In [ ]:
# 1개 IMAGE 폴더로 Train, Test dataset으로 나눠야 하므로, validation_split 사용
# Data augmentation(이미지 변형을 통한 데이터 증강)을 사용하지 않음
# validation 데이터 사이즈 입력 : validation_split=0.2 --> 20%
# (주의점) MobileNetV2에 인풋으로 사용하기 전에 전 처리하는 코드가 preprocess_input으로 함수화 되어 있습니다. 따라서 그대로 사용하시면 됩니다.
# 그래서 rescale 수행하지 않음

In [ ]:
image_datagen = ImageDataGenerator(
    # rescale=1./255,
    validation_split=0.2, # MobileNetV2의 경우, 아래 preprocess_input을 호출해서 리스케일 함
    preprocessing_function=preprocess_input
    # rotation_range=30,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # shear_range=0.1,
    # zoom_range=0.1,
    # horizontal_flip=True,
    # fill_mode='nearest'
)

## (4) 학습

In [ ]:
# 이미지 폴더 내 데이터를 읽고, 배치, 셔플하고 라벨링 수행
num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (224, 224, 3) # MovileNetV2의 기본 입력값을 위해 사이즈 변경
num_classes = 2

In [ ]:
# 이미지 폴더 내의 데이터  읽고 배치 , 셔플하고 labeling 수행
# 실제로 1개 IMAGE 폴더에서 Train 32000건, Test 8000건 dataset 만듬
# 2개 Class에 대한 라벨링(라벨인코딩,원핫인코딩) 수행
# MobileNetV2은 [96, 128, 160, 192, 224] 사이즈만 지원하므로 flow_from_directory 함수에서 사이즈 변경함
# subset = 'training' --> training_generator 생성
# subset = 'validation' --> test_generator 생성

# IMAGE 폴더 밑에 .ipynb_checkpoints 폴더 있을경우 폴더 삭제
#!rm -rf ./IMAGE/.ipynb_checkpoints

training_generator = image_datagen.flow_from_directory(
    './IMAGE', # 데이터 저장 경로? flow_from_directory를 써서 카테고리 별로 경로를 구분하지 않아도 됨
    batch_size=batch_size,
    target_size=(224, 224),
    class_mode='categorical', # 이진분류이긴 한데... 이상하다고 다중분류 적긴 하심 ( 원핫코드? 가 나와서.. 카테고리가 맞다고 하심 )
    shuffle=True,
    subset='training'
)

test_generator = image_datagen.flow_from_directory(
    './IMAGE',
    batch_size=batch_size,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

In [ ]:
# 사전 훈련된 모델 MobileNet V2에서 기본 모델을 생성합니다.
# 아래와 같은 형식을 Transfer Learning 사용하며 됩니다. 우리는 그냥 불러다 사용할줄 알면 됩니다.

base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [ ]:
# MobieNet V2 베이스 모델 고정하기
base_model.trainble = False

In [ ]:
# 모델 구축
x = base_model.output

x = tf.keras.layers.GlobalAveragePooling2D()(x) # GlobalAveragePooling2D: 3차원 텐서를 1차원으로 변환
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x) # num_classes: 2

model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function
              metrics=['accuracy'])             # Metrics / Accuracy

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

checkpoint_path = 'my_checkpoint.ckpt.weights.h5'
checkpoint = ModelCheckPoint(
    filepath = checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

lrReducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
# 학습
history = model.fit(
    training_generator,
    validation_data=test_generator,
    epochs=num_epochs, # num_epochs: 10
    batch_size=batch_size,
    callbacks=[es, checkpoint, lrReducer]
)

## (5) 결과 제출

In [ ]:
model.save("01073002902_3.h5")

In [ ]:
# test 폴더의 파일명으로 csv의 target column 만들기
target = []
result = []

for item in os.listdir(test): # test 데이터 경로
    target.append(item.split(".")[0]) # 확장자를 제외한 파일명만 추출

final = pd.DataFrame({"target": target, "result": result})

In [ ]:
result = []
answer_class = {0: 'Negative', 1: 'Positive'}
right = 0
total = 0

for image, label in zip(final['target'], final['result']):
    total += 1
    test_image_path = os.path.join(test, image + '.jpg')
    img = load_img(test_image_path, target_size=(224, 224)) # 파일 변환 사이즈만 알려주면 됨
    img_array = img_to_array(img) # 읽은 이미지를 Numpy 배열로변환 (224, 224, 3) 크기.
    img_array = np.expand_dims(img_array, axis=0) # 배열 차원 확장 (1, 224, 224, 3)

    # 이미지 데이터 전처리
    img_array = preprocess_input(img_array)

    # 예측
    predictions = model.predict(img_array)
    print(predictions)

    # 예측 결과
    predicted_class = np.argmax(predictions, axis=1)
    confidence = np.max(predictions, axis=1)

    # 클래스 인덱스를 클래스명으로 변환
    pred = answer_class[predicted_class[0]]
    result.append(pred)

In [ ]:
final['result'] = result
final.head()

In [ ]:
final.to_csv('01073002902_3.csv', index=False)

# 2. 다중분류_꽃사진분류

## (1) import

In [ ]:
from glob import glob

import zipfile
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 전처리, 학습
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## (2) 데이터 확인

In [ ]:
with zipfile.ZipFile(folder_path + '...images.zip', 'r') as zip_ref:
    zip_ref.extractall(folder_path)

In [ ]:
# train/test 이미지가 들어있는 폴더의 경로
train = '...images/train'
test = '...images/test'

In [ ]:
# 이미지가 분류되어 있지 않고, 파일명에 기입되어 있는 경우 사용하는 코드
if not os.path.exists('IMAGE'):
    os.mkdir('IMAGE')
if not os.path.exists('IMAGE/daisy'):
    os.mkdir('IMAGE/daisy')
if not os.path.exists('IMAGE/dandelion'):
    os.mkdir('IMAGE/dandelion')
if not os.path.exists('IMAGE/tulips'):
    os.mkdir('IMAGE/tulips')
if not os.path.exists('IMAGE/roses'):
    os.mkdir('IMAGE/roses')
if not os.path.exists('IMAGE/sunflowers'):
    os.mkdir('IMAGE/sunflowers')

for filename in os.listdir(train):
    if filename.endswith('.jpg'):
        label=filename.split("_")[-1].split(".")[0]
        if(label=="daisy"):
          shutil.copy(os.path.join(train, filename), os.path.join("IMAGE/daisy", filename))
        elif(label=="dandelion"):
          shutil.copy(os.path.join(train, filename), os.path.join("IMAGE/dandelion", filename))
        elif(label=="tulips"):
          shutil.copy(os.path.join(train, filename), os.path.join("IMAGE/tulips", filename))
        elif(label=="roses"):
          shutil.copy(os.path.join(train, filename), os.path.join("IMAGE/roses", filename))
        elif(label=="sunflowers"):
          shutil.copy(os.path.join(train, filename), os.path.join("IMAGE/sunflowers", filename))

In [ ]:
# daisy 폴더 안의 이미지 개수 !! 폴더 경로 수정 필요
!ls -l IMAGE/daisy | grep jpg | wc -l

In [ ]:
# 이미지 패스 지정
img_path = 'IMAGE'

## (3) 데이터 전처리

In [ ]:
image_datagen = ImageDataGenerator(
    # rescale=1./255,
    validation_split=0.2, # MobileNetV2의 경우, 아래 preprocess_input을 호출해서 리스케일 함
    preprocessing_function=preprocess_input
    # rotation_range=30,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # shear_range=0.1,
    # zoom_range=0.1,
    # horizontal_flip=True,
    # fill_mode='nearest'
)

### 1. 데이터 확인

In [ ]:
# class 이름 및 번호 매핑 확인
print(training_generator.class_indices)

In [ ]:
# 이미지 확인
batch_samples = next(iter(training_generator))

print('One Hot Encoding : ',batch_samples[1][0]) #label 배치의 첫번째 값
print(batch_samples[0][0].shape)  #이미지 배치의 첫번째 이미지
plt.imshow(batch_samples[0][0])
plt.show()

In [ ]:
# 이미지 크기 확인
print(batch_samples[0].shape)
print(batch_samples[1].shape)

## (4) 학습

In [ ]:
num_epochs = 4
batch_size =32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (224, 224, 3)  # 사이즈 확인
num_classes = 5    #  다섯 가지의 꽃 종류

In [ ]:
# IMAGE 폴더 밑에 .ipynb_checkpoints 폴더 있을경우 폴더 삭제
#!rm -rf ./IMAGE/.ipynb_checkpoints

training_generator = image_datagen.flow_from_directory(
    img_path, # 데이터 저장 경로? flow_from_directory를 써서 카테고리 별로 경로를 구분하지 않아도 됨
    batch_size=batch_size,
    target_size=(224, 224),
    class_mode='categorical', # 이진분류이긴 한데... 이상하다고 다중분류 적긴 하심 ( 원핫코드? 가 나와서.. 카테고리가 맞다고 하심 )
    shuffle=True,
    subset='training'
)

test_generator = image_datagen.flow_from_directory(
    img_path,
    batch_size=batch_size,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

In [ ]:
# 사전 훈련된 모델 MobileNet V2에서 기본 모델을 생성합니다.
# 아래와 같은 형식을 Transfer Learning 사용하며 됩니다. 우리는 그냥 불러다 사용할줄 알면 됩니다.

base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [ ]:
# MobieNet V2 베이스 모델 고정하기
base_model.trainble = False

In [ ]:
# 모델 구축
x = base_model.output

x = tf.keras.layers.GlobalAveragePooling2D()(x) # GlobalAveragePooling2D: 3차원 텐서를 1차원으로 변환
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x) # num_classes:5

model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.summary()

In [ ]:
# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function
              metrics=['accuracy'])             # Metrics / Accuracy

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

checkpoint_path = 'my_checkpoint.ckpt.weights.h5'
checkpoint = ModelCheckPoint(
    filepath = checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

lrReducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
# 학습
history = model.fit(
    training_generator,
    validation_data=test_generator,
    epochs=num_epochs, # num_epochs: 4
    batch_size=batch_size,
    callbacks=[es, checkpoint, lrReducer]
)

## (5) 그래프 확인

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

## (6) 결과 제출

In [ ]:
model.save("01073002902_3.h5")

In [ ]:
# test 폴더의 파일명으로 csv의 target column 만들기
target = []
result = []

for item in os.listdir(test): # test 데이터 경로
    target.append(item.split(".")[0]) # 확장자를 제외한 파일명만 추출

final = pd.DataFrame({"target": target, "result": result})

In [ ]:
result = []
answer_class = {0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}

In [ ]:
result = []
right = 0
total = 0

for image, label in zip(final['target'], final['result']):
    total += 1
    test_image_path = os.path.join(test, image + '.jpg')
    img = load_img(test_image_path, target_size=(224, 224)) # 파일 변환 사이즈만 알려주면 됨
    img_array = img_to_array(img) # 읽은 이미지를 Numpy 배열로변환 (224, 224, 3) 크기.
    img_array = np.expand_dims(img_array, axis=0) # 배열 차원 확장 (1, 224, 224, 3)

    # 이미지 데이터 전처리
    img_array = preprocess_input(img_array)

    # 예측
    predictions = model.predict(img_array)
    print(predictions)

    # 예측 결과
    predicted_class = np.argmax(predictions, axis=1)
    confidence = np.max(predictions, axis=1)

    # 클래스 인덱스를 클래스명으로 변환
    pred = answer_class[predicted_class[0]]
    result.append(pred)

In [ ]:
final['result'] = result
final.head()

In [ ]:
final.to_csv('01073002902_3.csv', index=False)